In [1]:
import numpy as np
from PIL import Image
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
import torchvision.transforms as transforms

In [2]:
transform = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5)),
    ]
)

In [4]:
train_data = torchvision.datasets.CIFAR10(root="./data", train=True, transform=transform, download=True)
test_data = torchvision.datasets.CIFAR10(root="./data", train=False, transform=transform, download=True)

train_loader = torch.utils.data.DataLoader(dataset=train_data, batch_size=32, shuffle=True, num_workers=10)
test_loader = torch.utils.data.DataLoader(dataset=test_data, batch_size=32, shuffle=True, num_workers=10)

In [6]:
image, label = train_data[0]
image.size()

torch.Size([3, 32, 32])

In [7]:
class_names = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

In [8]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()

        KERNE_SIZE= 5
        self.conv1 = nn.Conv2d(3, 12, KERNE_SIZE)
        self.conv2 = nn.Conv2d(12, 24, KERNE_SIZE)
        self.pool = nn.MaxPool2d(2,2)

        self.fc1 = nn.Linear(24 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 81)
        self.fc3 = nn.Linear(81, 10)

    def forward(self,x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))

        x = torch.flatten(x, 1)

        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        
        return x

In [9]:
net = NeuralNetwork()
loss_funtion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [11]:
for epoch in range(30):
    print(f"eğitim {epoch}...")

    running_loss = 0
    
    for i, data in enumerate(train_loader):
        inputs, labels = data

        optimizer.zero_grad()

        outputs = net(inputs)

        loss = loss_funtion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

print(f"loss: {running_loss / len(train_loader):.4}")

eğitim 0...
eğitim 1...
eğitim 2...
eğitim 3...
eğitim 4...
eğitim 5...
eğitim 6...
eğitim 7...
eğitim 8...
eğitim 9...
eğitim 10...
eğitim 11...
eğitim 12...
eğitim 13...
eğitim 14...
eğitim 15...
eğitim 16...
eğitim 17...
eğitim 18...
eğitim 19...
eğitim 20...
eğitim 21...
eğitim 22...
eğitim 23...
eğitim 24...
eğitim 25...
eğitim 26...
eğitim 27...
eğitim 28...
eğitim 29...
loss: 0.3972


In [12]:
torch.save(net.state_dict(), "eguitilmis_veri.pth")

In [13]:
net = NeuralNetwork()
net.load_state_dict(torch.load("eguitilmis_veri.pth"))

<All keys matched successfully>

In [17]:
correct = 0
total = 0

net.eval()

with torch.no_grad():
    for data in test_loader:
        images, labels = data
        outputs = net(images)
        _, pridicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (pridicted == labels).sum().item()


accuracy = 100 * correct / total

print(f"Doğruluk: {accuracy}")

Doğruluk: 68.4


In [19]:
new_trasform = transforms.Compose([
    transforms.Resize((32,32)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

def load_image(image_path):
    image = Image.open(image_path)
    image = new_trasform(image)
    image = image.unsqueeze(0)

    return image


image_path = ["kedi.png", "ucak.png"]
images = [load_image(img) for img in image_path]

net.eval()

with torch.no_grad():
    for image in images:
        output = net(image)
        _, pridicted = torch.max(output, 1)
        print(f"tahmin: {class_names[pridicted.item()]}")

tahmin: dog
tahmin: plane
